In [112]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense 



# 1. Traitement des données 

Importation du jeu de données et division en données classe et caractéristique

In [128]:
dt = pd.read_csv("/Users/mallorylecorre/Desktop/Dm_Ia/cars_data-class.csv")
print(dt.head())

X = dt.drop('Impact_Environnement', axis=1)
y = dt['Impact_Environnement']

   Taille_Auto   Poids_Auto  Acceleration  Annee_Fabrication    Kilometrage  \
0     3.790317  8869.946374      2.038688               2006  135542.067972   
1     3.375892  1126.174079      2.067411               2010   33710.551662   
2     4.844971  6100.019066      4.839676               2016  112977.279109   
3     2.316172  5771.509867      1.202574               2014   50139.727635   
4     2.618825  5676.666211      5.470806               2009  191188.750210   

   Nombre_Portes  Vitesse_Max  Prix_Assurance  Categorie_Voiture    Decote  \
0              5   195.289066      184.125577                  1  0.104588   
1              5   249.979656      858.935088                  3  0.346064   
2              5   235.763320      302.138838                  4  0.394582   
3              4   216.802020     1298.671966                  5  0.422048   
4              4   234.004505      702.626517                  2  0.279959   

   Impact_Environnement  
0                     1  
1   

In [129]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


On standardise les données pour mettre toute les variables à la même échelle.

In [130]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Distribution des données :

In [131]:
dt['Impact_Environnement'].value_counts()

1    537
3    385
2     42
0     36
Name: Impact_Environnement, dtype: int64

On peut voir que le jeu de données est déséquilibrée avec une sur-représentation de la classe 1.

On équilibre les données en sur-échantillonant avec SMOTE

In [132]:
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


On vérifie que le rééquillibrage a bien fonctionné

In [134]:
print("Nouvelle distribution :")
print(pd.Series(y_train).value_counts())

Nouvelle distribution :
0    28
1    28
2    28
3    28
Name: Impact_Environnement, dtype: int64


Les données sont équilibrées

# Création du modèle 

Création du modèle avec trois couches. plusieurs tests ont été effectué pour trouver l'architecture des couches qui donnent les meilleurs résultats. La fonction d'activation relu est utilisé pour ses avantages et une dernière couche softmax car on a une classificaiton multi-classes.

In [135]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

compilation du modèle en tuilisant une fonction de perte qui correspond à une classification multi-class

In [136]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Ajout d'une cross validatoin

Entrainement du modèle 

In [137]:
model.fit(X_train, y_train, epochs=200, batch_size=40, validation_split=0.2)


Epoch 1/200


3/3 [==============================] - 0s 34ms/step - loss: 1.3350 - accuracy: 0.3371 - val_loss: 1.9115 - val_accuracy: 0.0000e+00
Epoch 2/200
3/3 [==============================] - 0s 7ms/step - loss: 1.2898 - accuracy: 0.3483 - val_loss: 1.9321 - val_accuracy: 0.0000e+00
Epoch 3/200
3/3 [==============================] - 0s 7ms/step - loss: 1.2540 - accuracy: 0.3933 - val_loss: 1.9613 - val_accuracy: 0.0000e+00
Epoch 4/200
3/3 [==============================] - 0s 7ms/step - loss: 1.2211 - accuracy: 0.4157 - val_loss: 1.9833 - val_accuracy: 0.0000e+00
Epoch 5/200
3/3 [==============================] - 0s 7ms/step - loss: 1.1917 - accuracy: 0.4382 - val_loss: 2.0091 - val_accuracy: 0.0000e+00
Epoch 6/200
3/3 [==============================] - 0s 7ms/step - loss: 1.1644 - accuracy: 0.5056 - val_loss: 2.0369 - val_accuracy: 0.0000e+00
Epoch 7/200
3/3 [==============================] - 0s 7ms/step - loss: 1.1385 - accuracy: 0.5393 - val_loss: 2.0585 - val_accuracy: 0.0000e+00
Epoch 8/20

# Evaluation du modèle

In [138]:
y_pred_probabilities = model.predict(X_test)
y_pred = np.argmax(y_pred_probabilities, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

7/7 [==============================] - 0s 557us/step
Accuracy: 0.49


on a une accuracy de 0.80

In [139]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.61      0.68      0.64       109
           2       0.10      0.57      0.17         7
           3       0.91      0.26      0.41        76

    accuracy                           0.49       200
   macro avg       0.40      0.38      0.30       200
weighted avg       0.68      0.49      0.51       200



La classe 0 et la classe 2 n'ont pas du tout un bon score (0 % pour la classe 0), cela peut être du à leurs sous-reprrésentation dans le jeu de données de bases.